In [ ]:
from skimage import data
from skimage.util.dtype import dtype_range
from skimage.util import img_as_ubyte
from skimage import exposure
from skimage.morphology import disk
from skimage.filters import rank
import numpy as np

import cv2

import matplotlib.pyplot as plt

In [ ]:
def plot_img_and_hist(image, axes, bins=256):
    """Plot an image along with its histogram and cumulative histogram.

    """
    ax_img, ax_hist = axes
    ax_cdf = ax_hist.twinx()

    # Display image
    ax_img.imshow(image, cmap=plt.cm.gray)
    ax_img.set_axis_off()

    # Display histogram
    ax_hist.hist(image.ravel(), bins=bins)
    ax_hist.ticklabel_format(axis='y', style='scientific', scilimits=(0, 0))
    ax_hist.set_xlabel('Pixel intensity')

    xmin, xmax = dtype_range[image.dtype.type]
    ax_hist.set_xlim(xmin, xmax)

    # Display cumulative distribution
    img_cdf, bins = exposure.cumulative_distribution(image, bins)
    ax_cdf.plot(bins, img_cdf, 'r')

    return ax_img, ax_hist, ax_cdf

In [ ]:
img = cv2.imread('new_images/microcystis/images/raritan TD SRO 8112020 microcystis 2_lab.jpg')


In [ ]:
# Load an example image
img = img_as_ubyte()

# Global equalize
img_rescale = exposure.equalize_hist(img)

# # Equalization
selem = disk(30)
img_eq = rank.equalize(img, footprint=selem)


# Display results
fig = plt.figure(figsize=(8, 5))
axes = np.zeros((2, 3), dtype=np.object_)
axes[0, 0] = plt.subplot(2, 3, 1)
axes[0, 1] = plt.subplot(2, 3, 2, sharex=axes[0, 0], sharey=axes[0, 0])
axes[0, 2] = plt.subplot(2, 3, 3, sharex=axes[0, 0], sharey=axes[0, 0])
axes[1, 0] = plt.subplot(2, 3, 4)
axes[1, 1] = plt.subplot(2, 3, 5)
axes[1, 2] = plt.subplot(2, 3, 6)

ax_img, ax_hist, ax_cdf = plot_img_and_hist(img, axes[:, 0])
ax_img.set_title('Low contrast image')
ax_hist.set_ylabel('Number of pixels')

ax_img, ax_hist, ax_cdf = plot_img_and_hist(img_rescale, axes[:, 1])
ax_img.set_title('Global equalise')

ax_img, ax_hist, ax_cdf = plot_img_and_hist(img_eq, axes[:, 2])
ax_img.set_title('Local equalize')
ax_cdf.set_ylabel('Fraction of total intensity')


# prevent overlap of y-axis labels
fig.tight_layout()
plt.show()

In [ ]:
# Apply CLAHE to each color channel independently
# clahe = exposure.equalize_adapthist(img, clip_limit=0.5)



# display original and CLAHE images side by side
fig, axes = plt.subplots(1, 2, figsize=(8, 4))
ax = axes.ravel()
ax[0].imshow(img)
ax[0].set_title('Original')
ax[1].imshow(clahe)
ax[1].set_title('CLAHE')
for a in ax:
    a.axis('off')
fig.tight_layout()
plt.show()

In [ ]:
# img = cv2.imread('new_images/microcystis/images/microcystis-4585456-original.jpg')
# img = cv2.imread('b_ds/test/images/dolichospermum-21746007-original.jpg')
# img = cv2.imread('b_ds/test/images/aphanizomenon-102059231-original.jpg')
# img = cv2.imread('b_ds/test/images/NP_Detritus_032.jpg')
img = cv2.imread('b_ds/test/images/VEP1-cyano-march17_lab.jpg')


# resize img to 512x512

img = cv2.resize(img, (512, 512))

# do regular CLAHE

# convert RGB to BGR for opencv
img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)

lab_planes = list(cv2.split(lab))

clahe = cv2.createCLAHE(clipLimit=8, tileGridSize=(32,32))

lab_planes[0] = clahe.apply(lab_planes[0])

lab = cv2.merge(tuple(lab_planes))

bgr = cv2.cvtColor(lab, cv2.COLOR_LAB2BGR)

rgb = cv2.cvtColor(bgr, cv2.COLOR_BGR2RGB)
rgb = rgb[...,::-1]

# show original and CLAHE image matplotlib
fig, axes = plt.subplots(1, 2, figsize=(8, 4))
ax = axes.ravel()
ax[0].imshow(img)
ax[0].set_title('Original')
ax[1].imshow(bgr)
ax[1].set_title('CLAHE')


# cv2.imwrite('microcystis-4585456-original.jpg', rgb)

# save bgr image
# cv2.imwrite('new_images/microcystis/images/microcystis-4585456-original.png', bgr)

# img_gray = cv2.imread('b_ds/test/images/dolichospermum-21746007-original.jpg', 0)

# img_gray = cv2.resize(img_gray, (512, 512))

# ax[3].imshow(img_gray)
# ax[3].set_title('Grayscale')

# # do regular CLAHE

# clahe = cv2.createCLAHE(clipLimit=4, tileGridSize=(15,15))

# img = clahe.apply(img_gray)

# ax[4].imshow(img)
# ax[4].set_title('CLAHE Grayscale')

# for a in ax:
#     a.axis('off')
# fig.tight_layout()
# plt.show()

In [ ]:
# list of image file names
image_files = ['new_images/microcystis/images/microcystis-4585456-original.jpg',
               'b_ds/test/images/dolichospermum-21746007-original.jpg',
               'b_ds/test/images/aphanizomenon-102059231-original.jpg',
               'b_ds/test/images/NP_Detritus_032.jpg',
               'b_ds/test/images/VEP1-cyano-march17_lab.jpg']

# loop over images
for filename in image_files:
    # read image
    img = cv2.imread(filename)

    # resize img to 512x512
    img = cv2.resize(img, (512, 512))

    # convert RGB to BGR for opencv
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    # iterate over clipLimits and tileGridSizes

    fig, ax = plt.subplots()
    ax.imshow(img)
    ax.set_title('Original')
    ax.axis('off')
    plt.show()

    clip_limits = [4, 8]
    tile_grid_sizes = [(4, 4), (8, 8), (16, 16), (32, 32), (64, 64)]
    num = len(clip_limits) * len(tile_grid_sizes)
    fig, ax = plt.subplots(1, num, figsize=(20, 10))
    l = []
    for clip_limit in clip_limits:
        for tile_size in tile_grid_sizes:
            lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)

            lab_planes = list(cv2.split(lab))
            clahe = cv2.createCLAHE(clipLimit=clip_limit, tileGridSize=tile_size)
            lab_planes[0] = clahe.apply(lab_planes[0])
            lab = cv2.merge(tuple(lab_planes))
            bgr = cv2.cvtColor(lab, cv2.COLOR_LAB2BGR)
            rgb = cv2.cvtColor(bgr, cv2.COLOR_BGR2RGB)
            rgb = rgb[...,::-1]

            # plot image
            
            

            l.append([clip_limit, tile_size, rgb])


    for i in range(len(l)):
        ax[i].imshow(l[2])
        ax[i].set_title(f'CLAHE (clipLimit={l[0]}, tileGridSize={l[1]})')
        ax[i].axis('off')
    plt.show()

In [ ]:

# list of image file names
image_files = ['new_images/microcystis/images/microcystis-4585456-original.jpg',
               'b_ds/test/images/dolichospermum-21746007-original.jpg',
               'b_ds/test/images/aphanizomenon-102059231-original.jpg',
               'b_ds/test/images/NP_Detritus_032.jpg',
               'b_ds/test/images/VEP1-cyano-march17_lab.jpg',
               'b_ds/test/images/woronichinia-86823200-original.jpg']

# loop over images
for filename in image_files:
    # read image
    img = cv2.imread(filename)

    # resize img to 512x512
    img = cv2.resize(img, (512, 512))

    # convert RGB to BGR for opencv
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    # iterate over clipLimits and tileGridSizes

    fig, ax = plt.subplots(1, 1+len(clip_limits)*len(tile_grid_sizes), figsize=(30, 15))
    fig.subplots_adjust(wspace=0.5)

    ax[0].imshow(img)
    ax[0].set_title('Original', fontsize=6)
    ax[0].axis('off')
    
    clip_limits = [4, 8]
    tile_grid_sizes = [(4, 4), (8, 8), (16, 16), (32, 32), (64, 64)]
    i = 1
    for clip_limit in clip_limits:
        for tile_size in tile_grid_sizes:
            lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)

            lab_planes = list(cv2.split(lab))
            clahe = cv2.createCLAHE(clipLimit=clip_limit, tileGridSize=tile_size)
            lab_planes[0] = clahe.apply(lab_planes[0])
            lab = cv2.merge(tuple(lab_planes))
            bgr = cv2.cvtColor(lab, cv2.COLOR_LAB2BGR)
            rgb = cv2.cvtColor(bgr, cv2.COLOR_BGR2RGB)
            rgb = rgb[...,::-1]

            # plot image
            ax[i].imshow(rgb)
            ax[i].set_title(f'clipLimit={clip_limit} tileGridSize={tile_size}', fontsize=6)
            ax[i].axis('off')
            i += 1
    plt.show()

In [ ]:
# clip limit 4 and tile grid size 32
clip_limit = 4
tile_grid_size = (32, 32)